# Amazon SageMaker AutoPilot을 사용하여 고객 이탈을 예측
_**Using AutoPilot to Predict Mobile Customer Departure**_

---

---

Kernel `Python 3 (Data Science)` works well with this notebook.

## 내용

1. [소개](#소개)
1. [셋업](#셋업)
1. [데이터](#데이터)
1. [학습](#학습)
1. [결과](#결과)
1. [호스트](#호스트)
1. [클린업](#클린업)


---

## 소개

Amazon SageMaker Autopilot은 표 데이터셋을 위한 자동 기계 학습(일반적으로 AutoML) 솔루션입니다. SageMaker Autopilot은 자동 조종(이름 입력) 또는 사용자 안내와 함께, SageMaker Studio를 통해 코드 없이 또는 AWS SDK를 사용하여 다양한 방법으로 사용할 수 있습니다. 이 노트북은 AWS SDK를 사용하여 머신러닝 모델을 간단하게 만들고 배치합니다.

고객을 잃는 것은 어떤 사업에도 비용이 많이 듭니다. 불만족 고객을 조기에 파악하면 고객에게 숙박 인센티브를 제공할 수 있습니다. 이 노트북에서는 고객 이탈 예측이라고도 하는 불행한 고객을 자동으로 식별하기 위해 기계 학습(ML)을 사용하는 방법에 대해 설명합니다. 그러나 ML 모델은 완벽한 예측을 제공하는 경우가 거의 없기 때문에 이 노트북은 ML을 사용할 때의 재무 결과를 결정할 때 예측 오류의 상대적 비용을 통합하는 방법에 관한 것입니다.

우리는 이동전화 사업자를 두고 있는 우리 모두에게 친숙한 동요의 예를 사용합니다. 내 제공자에게서 항상 흠을 잡을 수 있을 것 같아요. 그리고 만약 제 공급자가 제가 떠날 생각을 하고 있다는 것을 안다면, 적시에 인센티브를 제공할 수 있을 것입니다. 항상 전화 업그레이드를 사용하거나 새로운 기능을 활성화할 수 있습니다. 그리고 저는 그냥 가만히 있을 수도 있습니다. 인센티브는 종종 고객을 잃고 재취득하는 것보다 훨씬 더 비용 효율적입니다.

---
## 셋업

_이 노트북은 ml.m4.xlarge 노트북 인스턴스에서 생성 및 테스트되었습니다._

먼저 다음을 지정합니다.

- 학습 및 모델 데이터에 사용할 S3 버킷 및 접두사입니다. 노트북 인스턴스, 학습 및 호스팅과 동일한 영역 내에 있어야 합니다.
- IAM 역할 arn은 학습 및 호스팅을 통해 데이터에 대한 액세스를 제공합니다. 이러한 파일을 만드는 방법은 설명서를 참조합니다. 노트북 인스턴스, 학습 및/또는 호스팅에 둘 이상의 역할이 필요한 경우 boto regexp를 적절한 전체 IAM 역할 arn 문자열로 교체해 주십시오.


In [ ]:
import sagemaker
import boto3
from sagemaker import get_execution_role

region = boto3.Session().region_name

session = sagemaker.Session()

# You can modify the following to use a bucket of your choosing
bucket = session.default_bucket()
prefix = 'sagemaker/DEMO-autopilot-churn'

role = get_execution_role()

# This is the client we will use to interact with SageMaker AutoPilot
sm = boto3.Session().client(service_name='sagemaker',region_name=region)

다음으로 나머지 연습에 필요한 파이썬 라이브러리를 가져올 것입니다.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
import sys
import time
import json
from IPython.display import display
from time import strftime, gmtime
import sagemaker
from sagemaker.predictor import csv_serializer

---
## 데이터

모바일 운영자는 고객이 결국 이 서비스를 계속 사용하게 된 과거 기록을 보유하고 있습니다. 이 과거 정보를 사용하여 학습이라는 프로세스를 사용하여 한 이동 통신사의 교란 ML 모델을 구성할 수 있습니다. 모델을 학습한 후에는 임의 고객의 프로필 정보(모델 학습을 위해 사용했던 것과 동일한 프로필 정보)를 모델에 전달하고, 모델이 이 고객의 이탈 여부를 예측하도록 할 수 있습니다. 물론, 우리는 이 모델이 실수를 할 것으로 예상합니다. 결국 미래를 예측하는 것은 까다로운 비즈니스입니다! 하지만 예측 오류에 대처하는 방법도 알려드리겠습니다.

NAT에서 사용하는 데이터셋은 공개적으로 사용할 수 있으며 [Discovering Knowledge in Data](https://www.amazon.com/dp/0470908742/)에서 언급되었습니다. 저자는 캘리포니아 대학교 기계 학습 데이터셋의 어바인 리포지토리에 기인합니다. 이제 이 데이터 집합을 다운로드하고 읽어 보겠습니다.


In [ ]:
!apt-get install unzip
!wget http://dataminingconsultant.com/DKD2e_data_sets.zip
!unzip -o DKD2e_data_sets.zip

### S3에 데이터셋 업로드

데이터 집합에서 자동 실행을 실행하기 전에 먼저 데이터 집합에 명백한 오류가 없는지 확인합니다. 자동 실행 프로세스는 시간이 오래 걸릴 수 있으며 일반적으로 작업을 시작하기 전에 데이터 집합을 검사하는 것이 좋습니다. 이 특정 데이터 집합은 크기가 작으므로 노트북 인스턴스 자체에서 검사할 수 있습니다. 노트북 인스턴스 메모리에 맞지 않는 대규모 데이터셋이 있는 경우 Apache Spark와 같은 빅데이터 분석 도구를 사용하여 오프라인에서 데이터셋을 검사합니다. [Deequ](https://github.com/awslabs/deequ)은 Apache Spark 위에 구축된 라이브러리로서 대용량 데이터셋에 대한 검사를 수행하는 데 도움이 될 수 있습니다. 오토파일럿은 데이터셋을 최대 5GB까지 처리할 수 있습니다.

판다의 데이터 프레임에서 데이터를 읽고 살펴봅니다.

In [ ]:
churn = pd.read_csv('./Data sets/churn.txt')
pd.set_option('display.max_columns', 500)
churn

현대적인 기준으로 볼 때, 비교적 작은 데이터 집합으로, 각 레코드는 미지의 미국 모바일 통신 사업자의 고객 프로파일을 설명하기 위해 21개의 속성을 사용하는 레코드가 고작 3,333개입니다. 특성은 다음과 같습니다.

- `State`: OH 또는 NJ와 같은 두 글자의 약어로 고객이 거주하는 미국 상태입니다.
- `Account Length`: 이 계정이 활성화된 일 수입니다.
- `Area Code`: 해당 고객 전화 번호의 지역 번호 세 자리입니다.
- `Phone`: 남은 7자리 전화번호입니다.
- `Int’l Plan`: 고객이 국제 통화 계획을 가지고 있는지 여부: 예/아니요.
- `VMail Plan`: 고객이 음성 메일 기능을 가지고 있는지 여부: 예/아니요.
- `VMail Message`: 아마도 월 평균 음성 메일 메시지 수입니다.
- `Day Mins`: 하루 동안 사용된 총 통화 시간(분)입니다.
- `Day Calls`: 하루 동안의 총 통화 횟수입니다.
- `Day Charge`: 주간 통화 요금이 청구됩니다.
- `Eve Mins, Eve Calls, Eve Charge`: 저녁 시간 동안 걸려온 전화의 청구 비용입니다.
- `Night Mins`, `Night Calls`, `Night Charge`: 야간 통화 요금이 청구됩니다.
- `Intl Mins`, `Intl Calls`, `Intl Charge`: 국제전화 요금 청구입니다.
- `CustServ Calls`: 고객 서비스에 대한 통화 횟수입니다.
- `Churn?`: 고객이 서비스를 떠났는지 여부: 참/거짓입니다.

마지막 속성인 `Churn?`은 타겟 속성으로 ML 모델이 예측하기를 원하는 속성입니다.


### 모델에 대한 추론을 호출하기 위해 일부 데이터를 예약합니다.

데이터를 train & test 분할로 나눕니다. train 분할은 SageMaker Autopilot에서 사용됩니다. test 분할은 제안된 모델을 사용하여 추론을 수행하도록 예약되어 있습니다.

In [ ]:
train_data = churn.sample(frac=0.8,random_state=200)

test_data = churn.drop(train_data.index)

test_data_no_target = test_data.drop(columns=['Churn?'])

이제 이 파일들을 S3에 업로드하겠습니다.

In [ ]:
train_file = 'train_data.csv';
train_data.to_csv(train_file, index=False, header=True)
train_data_s3_path = session.upload_data(path=train_file, key_prefix=prefix + "/train")
print('Train data uploaded to: ' + train_data_s3_path)

test_file = 'test_data.csv';
test_data_no_target.to_csv(test_file, index=False, header=False)
test_data_s3_path = session.upload_data(path=test_file, key_prefix=prefix + "/test")
print('Test data uploaded to: ' + test_data_s3_path)

---
## SageMaker 자동 실행 작업을 설정합니다.<a name="학습"></a>

데이터 집합을 Amazon S3에 업로드한 후 Autopilot을 호출하여 이 데이터 집합에서 모델을 학습할 수 있는 최상의 ML 파이프라인을 찾을 수 있습니다.

Autopilot 작업을 호출하는 데 필요한 입력은 다음과 같습니다.
* 입력 데이터 집합 및 모든 출력 아티팩트에 대한 Amazon S3 위치입니다.
* 예측하고자 하는 데이터 집합의 열 이름입니다(이 경우 `Churn?`)입니다.
* IAM 역할입니다.

현재 Autopilot은 CSV 형식의 표 형식의 데이터셋만 지원합니다. 모든 파일에 헤더 행이 있어야 하거나 이름별로 알파벳/단어 순서로 정렬된 데이터 집합의 첫 번째 파일에는 헤더 행이 있어야 합니다.


In [ ]:
input_data_config = [{
      'DataSource': {
        'S3DataSource': {
          'S3DataType': 'S3Prefix',
          'S3Uri': 's3://{}/{}/train'.format(bucket,prefix)
        }
      },
      'TargetAttributeName': 'Churn?'
    }
  ]

output_data_config = {
    'S3OutputPath': 's3://{}/{}/output'.format(bucket,prefix)
  }


또한 데이터 집합으로 해결할 문제 유형(`Regression, MulticlassClassification, BinaryClassification`)을 지정할 수 있습니다. 확실하지 않은 경우 SageMaker Autopilot은 대상 열(예측할 열)의 통계를 기반으로 문제 유형을 추정합니다.

왜냐하면 그 목표 속성인 ```Churn?```는 이진수이며 우리 모델은 binary classification을 수행할 것이다. 이 예에서는 AutoPilot이 문제의 유형을 추론하도록 하겠습니다.

파이프라인 평가 또는 지원자의 최대 수를 제공하거나(하나의 파이프라인 평가를 후보 모델을 생성하므로 `Candidate`라고 함) 전체 자동 실행 작업에 할당된 총 시간을 제공하여 SageMaker 자동 실행 작업의 실행 시간을 제한할 수 있는 옵션이 있습니다. 기본 설정에서 이 작업을 실행하는 데 약 4시간이 걸립니다. 이는 Autopilot이 최적의 학습 매개변수를 찾기 위해 사용하는 탐색 프로세스의 특성 때문에 실행마다 다릅니다.

### SageMaker AutoPilot 작업을 시작합니다.<a name="Launching"></a>

이제 `create_auto_ml_job` API를 호출하여 Autopilot 작업을 시작할 수 있습니다. 우리는 지원자 수를 20명으로 제한하여 몇 분 안에 작업이 완료될 수 있도록 합니다.


In [ ]:
from time import gmtime, strftime, sleep
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())

auto_ml_job_name = 'automl-churn-' + timestamp_suffix
print('AutoMLJobName: ' + auto_ml_job_name)

sm.create_auto_ml_job(AutoMLJobName=auto_ml_job_name,
                      InputDataConfig=input_data_config,
                      OutputDataConfig=output_data_config,
                      AutoMLJobConfig={'CompletionCriteria':
                                       {'MaxCandidates': 20}
                                      },
                      RoleArn=role)

### SageMaker AutoPilot을 추적<a name="Tracking"></a>
SageMaker 자동 실행 작업은 다음과 같은 상위 단계로 구성됩니다.
* 데이터 분석: 데이터셋을 분석하고 Autopilot을 사용하면 데이터셋에서 시도해야 하는 ML 파이프라인 목록이 나타납니다. 데이터 집합은 또한 열차 및 유효성 검사 세트로 분할됩니다.
* Feature Engineering(기능 엔지니어링) - Autopilot(자동 조종)은 데이터셋의 개별 기능뿐 아니라 Aggregate 레벨에서도 기능 변환을 수행합니다.
* Model Tuning(모델 조정) - 학습 알고리즘(파이프라인의 마지막 단계)에 대한 최적의 하이퍼 파라미터와 함께 최고 성능의 파이프라인을 선택합니다.


In [ ]:
print ('JobStatus - Secondary Status')
print('------------------------------')


describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
print (describe_response['AutoMLJobStatus'] + " - " + describe_response['AutoMLJobSecondaryStatus'])
job_run_status = describe_response['AutoMLJobStatus']
    
while job_run_status not in ('Failed', 'Completed', 'Stopped'):
    describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    job_run_status = describe_response['AutoMLJobStatus']
    
    print (describe_response['AutoMLJobStatus'] + " - " + describe_response['AutoMLJobSecondaryStatus'])
    sleep(30)

---
## 결과

이제 descript_auto_ml_job API를 사용하여 SageMaker Autopilot 작업에서 선택한 최상의 지원자를 찾습니다.

In [ ]:
best_candidate = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)['BestCandidate']
best_candidate_name = best_candidate['CandidateName']
print(best_candidate)
print('\n')
print("CandidateName: " + best_candidate_name)
print("FinalAutoMLJobObjectiveMetricName: " + best_candidate['FinalAutoMLJobObjectiveMetric']['MetricName'])
print("FinalAutoMLJobObjectiveMetricValue: " + str(best_candidate['FinalAutoMLJobObjectiveMetric']['Value']))

관련된 알고리즘의 랜덤성으로 인해, 서로 다른 런은 약간 다른 결과를 제공하지만 정확도는 $93\%$ 이상이어야 하므로 좋은 결과입니다.

---
## 호스트

알고리즘을 학습했으므로 모델을 만들어 호스트된 엔드포인트에 배포하겠습니다.

In [ ]:
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())
model_name = best_candidate_name + timestamp_suffix + "-model"
model_arn = sm.create_model(Containers=best_candidate['InferenceContainers'],
                            ModelName=model_name,
                            ExecutionRoleArn=role)

epc_name = best_candidate_name + timestamp_suffix + "-epc"
ep_config = sm.create_endpoint_config(EndpointConfigName = epc_name,
                                      ProductionVariants=[{'InstanceType': 'ml.m5.2xlarge',
                                                           'InitialInstanceCount': 1,
                                                           'ModelName': model_name,
                                                           'VariantName': 'main'}])

ep_name = best_candidate_name + timestamp_suffix + "-ep"
create_endpoint_response = sm.create_endpoint(EndpointName=ep_name,
                                              EndpointConfigName=epc_name)

In [ ]:
sm.get_waiter('endpoint_in_service').wait(EndpointName=ep_name)

### 평가

이제 호스트된 엔드포인트가 실행 중이므로, 단순히 http POST 요청을 함으로써 모델에서 실시간 예측을 매우 쉽게 할 수 있습니다. 그러나 먼저 'test_data' NumPy 어레이를 끝점 뒤쪽에 있는 모델에 전달하기 위한 직렬화 장치와 탈세리알라이저를 설정해야 합니다.

In [ ]:
from io import StringIO
from sagemaker.predictor import RealTimePredictor
from sagemaker.content_types import CONTENT_TYPE_CSV


predictor = RealTimePredictor(
    endpoint=ep_name,
    sagemaker_session=session,
    content_type=CONTENT_TYPE_CSV,
    accept=CONTENT_TYPE_CSV)

# Remove the target column from the test data
test_data_inference = test_data.drop('Churn?', axis=1)

# Obtain predictions from SageMaker endpoint
prediction = predictor.predict(test_data_inference.to_csv(sep=',', header=False, index=False)).decode('utf-8')

# Load prediction in pandas and compare to ground truth
prediction_df = pd.read_csv(StringIO(prediction), header=None)
accuracy = (test_data.reset_index()['Churn?'] == prediction_df[0]).sum() / len(test_data_inference)
print('Accuracy: {}'.format(accuracy))

---
## 클린업

자동 실행 작업은 데이터 집합 분할, 사전 처리 스크립트 또는 사전 처리된 데이터 등과 같은 많은 기본 아티팩트를 생성합니다. 이 코드는 컴파일되지 않은 경우 해당 코드를 삭제합니다. 이 작업을 수행하면 생성된 모든 모델과 자동 생성된 노트북도 삭제됩니다.

In [ ]:
#s3 = boto3.resource('s3')
#s3_bucket = s3.Bucket(bucket)

#job_outputs_prefix = '{}/output/{}'.format(prefix, auto_ml_job_name)
#s3_bucket.objects.filter(Prefix=job_outputs_prefix).delete()

마지막으로 엔드포인트 및 관련 리소스를 삭제합니다.

In [ ]:
sm.delete_endpoint(EndpointName=ep_name)
sm.delete_endpoint_config(EndpointConfigName=epc_name)
sm.delete_model(ModelName=model_name)